In [1]:
import json
import pandas as pd
from sentence_transformers import SentenceTransformer
from sklearn.manifold import TSNE
from sklearn.cluster import AgglomerativeClustering
import plotly.express as px
import numpy as np

/Users/hyegang/Desktop/졸업논문/.venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
#load data
filepath = "results_prompt4.json"
with open(filepath) as f:
    data = json.load(f)


In [6]:
data

{'0': {'turn_1': {'turn_idx': 1,
   'speaker': 'user',
   'utterance': 'i need a place to dine in the center thats expensive',
   'dialogue_acts': {'Request': [['restaurant', 'area', 'center'],
     ['restaurant', 'price', 'expensive']]},
   'belief_state': [{'domain': 'restaurant',
     'slots': [['area', 'center'], ['price', 'expensive']]}],
   'transitions': {'state_1': [],
    'action': 'Request',
    'state_2': [{'domain': 'restaurant',
      'slots': [['area', 'center'], ['price', 'expensive']]}]},
   'reward': 'NA'},
  'turn_2': {'turn_idx': 2,
   'speaker': 'system',
   'utterance': 'I have several options for you; do you prefer African, Asian, or British food?',
   'dialogue_acts': {'Offer': [['restaurant', 'food', 'African'],
     ['restaurant', 'food', 'Asian'],
     ['restaurant', 'food', 'British']]},
   'belief_state': [{'domain': 'restaurant',
     'slots': [['area', 'center'], ['price', 'expensive']]}],
   'transitions': {'state_1': [{'domain': 'restaurant',
      'slot

In [7]:
import json

# Load data
filepath = "results_prompt4.json"
with open(filepath) as f:
    data = json.load(f)

def extract_state_action_sequences(data):
    all_sequences = []
    
    # Process each dialogue
    for dialogue_id, dialogue in data.items():
        # Get all turns sorted by index
        turns = []
        for key in dialogue:
            if key.startswith('turn_'):
                turn_idx = int(key.split('_')[1])
                turns.append((turn_idx, dialogue[key]))
        turns.sort()
        
        # Track state changes across turns
        sequence_parts = []
        
        for i, (turn_idx, turn) in enumerate(turns):
            transitions = turn.get('transitions', {})
            state_1 = transitions.get('state_1', [])
            action = transitions.get('action', 'NONE')
            state_2 = transitions.get('state_2', [])
            
            # Skip if no state_2 or it's the same as previous
            if not state_2:
                continue
                
            # Process all domains in state_2
            domain_slots = {}
            
            for domain_data in state_2:
                domain = domain_data.get('domain')
                if not domain or domain == 'context':  # Skip context domain
                    continue
                    
                for slot_name, slot_value in domain_data.get('slots', []):
                    domain_slots[f"{domain}-{slot_name}"] = slot_value
            
            if domain_slots:
                # Format the sequence part
                slot_parts = []
                for slot_key, slot_value in domain_slots.items():
                    domain = slot_key.split('-')[0]
                    action_prefix = f"FIND_{domain.upper()}" if action in ['Request', 'Inform'] else "NONE"
                    slot_parts.append(f"{action_prefix}({slot_key}={slot_value})")
                
                if slot_parts:
                    sequence_part = " + ".join(slot_parts)
                    
                    # Add to sequence if different from previous
                    if not sequence_parts or sequence_parts[-1] != sequence_part:
                        sequence_parts.append(sequence_part)
        
        if sequence_parts:
            all_sequences.append(" → ".join(sequence_parts))
    
    return all_sequences

# Extract and print sequences
sequences = extract_state_action_sequences(data)
for i, seq in enumerate(sequences):
    if seq:  # Only print non-empty sequences
        print(f"Dialogue {i+1}:")
        print(seq)
        print()

# Count sequences
print(f"Total sequences extracted: {len([s for s in sequences if s])}")

Dialogue 1:
FIND_RESTAURANT(restaurant-area=center) + FIND_RESTAURANT(restaurant-price=expensive) → NONE(restaurant-area=center) + NONE(restaurant-price=expensive) → FIND_RESTAURANT(restaurant-area=center) + FIND_RESTAURANT(restaurant-price=expensive) + FIND_RESTAURANT(restaurant-food=any) → FIND_RESTAURANT(restaurant-area=center) + FIND_RESTAURANT(restaurant-price=expensive) + FIND_RESTAURANT(restaurant-food=African) + FIND_RESTAURANT(restaurant-name=Bedouin) → FIND_RESTAURANT(restaurant-area=center) + FIND_RESTAURANT(restaurant-price=expensive) + FIND_RESTAURANT(restaurant-food=African) + FIND_RESTAURANT(restaurant-name=Bedouin) + FIND_HOTEL(hotel-price=expensive) → FIND_RESTAURANT(restaurant-area=center) + FIND_RESTAURANT(restaurant-price=expensive) + FIND_RESTAURANT(restaurant-food=African) + FIND_RESTAURANT(restaurant-name=Bedouin) + FIND_RESTAURANT(restaurant-phone=01223367660) + FIND_HOTEL(hotel-price=expensive) + FIND_HOTEL(hotel-name=University Arms Hotel) + FIND_HOTEL(hotel-a

In [8]:
import json

# Load data
filepath = "results_prompt4.json"
with open(filepath) as f:
    data = json.load(f)

def extract_state_action_sequences(data):
    all_sequences = []
    
    # Process each dialogue
    for dialogue_id, dialogue in data.items():
        dialogue_sequence = []
        current_state = {}
        
        # Get all turns sorted by index
        turns = []
        for key in dialogue:
            if key.startswith('turn_'):
                turn_idx = int(key.split('_')[1])
                turns.append((turn_idx, dialogue[key]))
        turns.sort()
        
        for turn_idx, turn in turns:
            transitions = turn.get('transitions', {})
            action = transitions.get('action', 'NONE')
            state_2 = transitions.get('state_2', [])
            
            # Extract all slots from all domains
            turn_state = {}
            for domain_data in state_2:
                domain = domain_data.get('domain')
                if domain and domain != 'context':  # Skip context domain
                    for slot_name, slot_value in domain_data.get('slots', []):
                        key = f"{domain}-{slot_name}"
                        turn_state[key] = slot_value
            
            # Format based on action and state
            if turn_state:
                formatted_parts = []
                for slot_key, slot_value in sorted(turn_state.items()):
                    domain = slot_key.split('-')[0]
                    action_prefix = f"FIND_{domain.upper()}" if action in ['Request', 'Inform'] else "NONE"
                    formatted_parts.append(f"{action_prefix}({slot_key}={slot_value})")
                
                state_str = " + ".join(formatted_parts)
                if state_str and (not dialogue_sequence or dialogue_sequence[-1] != state_str):
                    dialogue_sequence.append(state_str)
        
        if dialogue_sequence:
            all_sequences.append(" → ".join(dialogue_sequence))
    
    return all_sequences

# Extract all sequences
sequences = extract_state_action_sequences(data)

# Print results
for i, sequence in enumerate(sequences):
    if sequence:
        print(f"Dialogue {i+1}:")
        print(sequence)
        print()

print(f"Total sequences extracted: {len([s for s in sequences if s])}")

Dialogue 1:
FIND_RESTAURANT(restaurant-area=center) + FIND_RESTAURANT(restaurant-price=expensive) → NONE(restaurant-area=center) + NONE(restaurant-price=expensive) → FIND_RESTAURANT(restaurant-area=center) + FIND_RESTAURANT(restaurant-food=any) + FIND_RESTAURANT(restaurant-price=expensive) → FIND_RESTAURANT(restaurant-area=center) + FIND_RESTAURANT(restaurant-food=African) + FIND_RESTAURANT(restaurant-name=Bedouin) + FIND_RESTAURANT(restaurant-price=expensive) → FIND_HOTEL(hotel-price=expensive) + FIND_RESTAURANT(restaurant-area=center) + FIND_RESTAURANT(restaurant-food=African) + FIND_RESTAURANT(restaurant-name=Bedouin) + FIND_RESTAURANT(restaurant-price=expensive) → FIND_HOTEL(hotel-area=center) + FIND_HOTEL(hotel-name=University Arms Hotel) + FIND_HOTEL(hotel-price=expensive) + FIND_RESTAURANT(restaurant-area=center) + FIND_RESTAURANT(restaurant-food=African) + FIND_RESTAURANT(restaurant-name=Bedouin) + FIND_RESTAURANT(restaurant-phone=01223367660) + FIND_RESTAURANT(restaurant-price

In [ ]:
#embedding

In [ ]:
#similarity calculation

In [ ]:
#t-SNE

In [ ]:
#visualize